In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

In [3]:
data_dir

'./datasets/flower_photos'

In [4]:

import pathlib   # pathlib is used mainly for doing operations on path of a file or in a file or directory like
                 # fetching the files from a directory with only .txt/.jpeg extensions
data_dir=pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [5]:
len(list(data_dir.glob('*/*.jpg')))
'''You can also use the ** pattern to perform a recursive search through subdirectories.
 For example, if you want to find all text files in the specified directory and its subdirectories, you can use .glob("**/*.txt")
 '''

'You can also use the ** pattern to perform a recursive search through subdirectories.\n For example, if you want to find all text files in the specified directory and its subdirectories, you can use .glob("**/*.txt")\n '

In [6]:
flower_images_dict ={
    'roses':list(data_dir.glob('roses/*')),
    'tulips':list(data_dir.glob('tulips/*')),
    'dandelion':list(data_dir.glob('dandelion/*')),
    'sunflower':list(data_dir.glob('sunflower/*')),
    'daisy':list(data_dir.glob('daisy/*')),
}

In [7]:
flower_labels_dict={
    'roses':0,
    'tulips':1,
    'dandelion':2,
    'sunflower':3,
    'daisy':4
}

In [8]:
img=cv2.imread(str(flower_images_dict['roses'][0]))
img.shape

(333, 500, 3)

In [9]:
x,y=[],[]
for flower_name, images in flower_images_dict.items():
  for image in images:
    img=cv2.imread(str(image))
    resized_img=cv2.resize(img,(224,224))
    x.append(resized_img)
    y.append(flower_labels_dict[flower_name])

In [10]:
x=np.array(x)
y=np.array(y)

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [12]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2228, 224, 224, 3)
(2228,)
(743, 224, 224, 3)
(743,)


In [13]:
#scaling
X_train_scaled=x_train/255
y_train_scaled=y_train/255
x_test_scaled=x_test/255
y_test_scaled=y_test/255

In [14]:
num_classes = 5

# data_augmentation = keras.Sequential(
#   [
#     layers.experimental.preprocessing.RandomFlip("horizontal",
#                                                  input_shape=(180,
#                                                               180,
#                                                               3)),
#     layers.experimental.preprocessing.RandomRotation(0.1),
#     layers.experimental.preprocessing.RandomZoom(0.1),
#   ]
# )

# model = Sequential([
#     data_augmentation,
#     layers.Conv2D(16, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(32, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Dropout(0.2),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(num_classes,activation='softmax')
# ])

# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# model.fit(X_train_scaled, y_train, epochs=30)


In [15]:

#  model.evaluate(x_test_scaled,y_test)
#  # overfitting - data augmentation

In [16]:
# now using transfer learning training the same model, using mobilnet_v2 classification model

import tensorflow_hub as hub
import PIL.Image as Image


IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])

In [17]:
image_labels=[]

with open('ImageNetLabels.txt','r') as f:
  image_labels = f.read().splitlines()
image_labels[:5]

['background', 'tench', 'goldfish', 'great white shark', 'tiger shark']

In [18]:
predicted= classifier.predict(np.array([x[0],x[1],x[2],x[3],x[4]]))
predicted=np.argmax(predicted,axis=1)
predicted

1/1 [==============================] - 7s 7s/step


array([795, 795, 722, 795, 795])

In [19]:
print(image_labels[795],image_labels[722])

shower curtain pillow


In [20]:
# plt.imshow(x[0])

In [21]:
# plt.imshow(x[1])

In [22]:
# plt.imshow(x[2])

In [23]:
feature_extractor_model="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_model_wo_top_layer = hub.KerasLayer(
    feature_extractor_model,input_shape=(224,224,3), trainable=False
)

In [24]:
model= tf.keras.Sequential([
    pretrained_model_wo_top_layer,
    tf.keras.layers.Dense(num_classes)
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [25]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
70/70 [==============================] - 9s 50ms/step - loss: 0.7487 - accuracy: 0.7096
Epoch 2/10
70/70 [==============================] - 3s 41ms/step - loss: 0.3564 - accuracy: 0.8703
Epoch 3/10
70/70 [==============================] - 3s 41ms/step - loss: 0.2827 - accuracy: 0.9071
Epoch 4/10
70/70 [==============================] - 3s 41ms/step - loss: 0.2341 - accuracy: 0.9237
Epoch 5/10
70/70 [==============================] - 3s 42ms/step - loss: 0.2094 - accuracy: 0.9331
Epoch 6/10
70/70 [==============================] - 3s 41ms/step - loss: 0.1778 - accuracy: 0.9475
Epoch 7/10
70/70 [==============================] - 3s 41ms/step - loss: 0.1614 - accuracy: 0.9560
Epoch 8/10
70/70 [==============================] - 3s 41ms/step - loss: 0.1424 - accuracy: 0.9650
Epoch 9/10
70/70 [==============================] - 3s 42ms/step - loss: 0.1285 - accuracy: 0.9663
Epoch 10/10
70/70 [==============================] - 3s 41ms/step - loss: 0.1196 - accuracy: 0.9708


In [26]:
model.evaluate(x_test_scaled,y_test)

24/24 [==============================] - 2s 59ms/step - loss: 0.2685 - accuracy: 0.9004


[0.2684653103351593, 0.9004037976264954]